In [12]:
import sys
import os
sys.path.append("../util/")
import MapRegion
import text_util
import json

In [13]:
def process_tweet(tweet_json):
    if (tweet_json["doc"]["coordinates"] is not None):
        submitted_json = {}
        submitted_json['id'] = tweet_json["doc"]["id"]
        submitted_json['created_at'] = tweet_json["doc"]["created_at"]
        submitted_json['text'] = text_util.preprocess_tweet(tweet_json["doc"]["text"])
        submitted_json['coordinates'] = [tweet_json["doc"]["coordinates"]["coordinates"][1],
                                        tweet_json["doc"]["coordinates"]["coordinates"][0]]
        
        submitted_json['region'] = MapRegion.getCoordinateRegion(submitted_json['coordinates'])
        if (submitted_json['text'] != "" and submitted_json['region'] is not None):
            print(json.dumps(submitted_json))

In [14]:
def parseJson(tweet_row):
    try:
        tweet_json = json.loads(line[:-3])
        return tweet_json
    except:
        try:
            tweet_json = json.loads(line[:-2])
            return tweet_json
        except:
            return None


In [16]:
with open("./teacher_db/twitter_sydney_1_1_1_2.json","r") as f:
    for index, line in enumerate(f):
        if (index > 1000):
            break
        tweet_json = parseJson(line)
        if tweet_json is not None:
            process_tweet(tweet_json)

{"text": "lots of great memories despite a challenging year new", "created_at": "Mon Jan 01 00:17:33 +0000 2018", "region": "Sydney (C)", "id": 947622787111415800, "coordinates": [-33.8833, 151.225]}
